In [1]:
from bs4 import BeautifulSoup as bs
from urllib import request
import os
import re

In [2]:
# 현재 상영 중인 영화 목록을 알려주는 url을 바탕으로 bsObject 생성,
# 현재 상영 영화 목록을 담고 있는 ul태그를 movies에 담는다!
source_list = request.urlopen('https://movie.naver.com/movie/running/current.nhn?order=reserve')
soup= bs(source_list, 'html.parser')
movies = soup.find('ul', {'class':'lst_detail_t1'})

In [3]:
# 현재 상영 중인 영화들의 코드를 담아줄 list를 생성한다.
code_list = []

# 각 영화는 ul 태그 내(movies) li 태그로 열거되어 있으며,
# li 내부의 a태그를 통해 각 영화의 코드를 알아낼 수 있다!
for movie in movies.find_all('li'):
    movie_url = movie.a['href']    # ex) href="/movie/bi/mi/basic.nhn?code=132623"
    code = movie_url.split('=')[-1]
    # print(code)
    code_list.append(code)

132623
180425
167651
182360
179974
177374
167099
182206
148909
171539
172145
172143
182407
167054
174830
181959
170953
181717
163008
183104
167616
109193
174730
158873
181711
178097
154288
179318
160837
170286
179033
179694
181554
167648
174832
175051
151151
144379
180992
168740
156464
70617
181409
167629
152694
168743
168050
159070
174297
182841
140110
178483
144318
147065
164335
49687
119552
171725
53047
152632
66598
167699
52991
153297
112082
168744
154983
179666
98446
98447
174329
149479
174296
162229
21931
169262
86958
154433
148647
155123
182032
181544


In [4]:
# single에서 써준 그대로 해주면 된다!
for code in code_list:
    # 우선 code만으로는 어떤 영화에서 작업하는지 모르기 때문에 title을 추출해낸다.
    source_title = request.urlopen(f'https://movie.naver.com/movie/bi/mi/basic.nhn?code={code}').read()
    soup = bs(source_title, 'html.parser')
    title = soup.find('h3', {'class':'h_movie'}).a.text
    
    # 혹시나 디렉터리 이름에 부적합한 기호가 있는 경우 지워준다!
    title = re.sub('[\/:*?"<>|]', '', title)
    print(f'working on {title}')
    
    # 총 몇 개의 (개봉 후) 평점이 있는지 알아본다!
    total = soup.find('div',{'class':'score_total'})
    # 개중에는 평점이 없는 경우도 있다
    # 이런 경우에는 그냥 다음 영화로!
    if total == None:
        print('no replies!')
        continue
    total = total.find_all('em')[-1].text
    total = total.strip()
    total = int(re.sub(',','',total))
    print(f'We have {total} replies!')
    
    # 총 페이지수를 가늠해보자!
    if int(total) % 10 == 0:
        pages = int(total) // 10
    else:
        pages = int(total) // 10 +1
    
    result_list = []
    
    for page in range(pages):
        # HTTPError: HTTP Error 502: Bad Gateway
        if (page + 1)%100 ==0:
            print(f'current : {page + 1} of {pages}')
        # sleep(0.01)
        home = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={code}&type=after&page={page+1}'
        source = request.urlopen(home).read()
        soup = bs(source, 'lxml')
        stars = soup.find_all('div', {'class':'star_score'})[1:]   # 처음에 사용자 입장에서 별점 주는 칸도 있어서 11개가 나옴!
        replies = soup.find_all('div', {'class':"score_reple"})
        for s, r in zip(stars, replies):
            reply = r.p.text
            if r.p.span:
                reply = reply[3:]
            result = f'{s.em.text}:{reply}\n'
            result_list.append(result)
        
    with open(f'./movies/{title}.txt', 'w', encoding = 'utf-8',) as f:
        f.writelines(set(result_list))    # set으로 하여 중복된 값 없애줌!

working on 극한직업
We have 44112 replies!
current : 100 of 4412
current : 200 of 4412
current : 300 of 4412
current : 400 of 4412
current : 500 of 4412
current : 600 of 4412
current : 700 of 4412
current : 800 of 4412
current : 900 of 4412
current : 1000 of 4412
current : 1100 of 4412
current : 1200 of 4412
current : 1300 of 4412
current : 1400 of 4412
current : 1500 of 4412
current : 1600 of 4412
current : 1700 of 4412
current : 1800 of 4412
current : 1900 of 4412
current : 2000 of 4412
current : 2100 of 4412
current : 2200 of 4412
current : 2300 of 4412
current : 2400 of 4412
current : 2500 of 4412
current : 2600 of 4412
current : 2700 of 4412
current : 2800 of 4412
current : 2900 of 4412
current : 3000 of 4412
current : 3100 of 4412
current : 3200 of 4412
current : 3300 of 4412
current : 3400 of 4412
current : 3500 of 4412
current : 3600 of 4412
current : 3700 of 4412
current : 3800 of 4412
current : 3900 of 4412
current : 4000 of 4412
current : 4100 of 4412
current : 4200 of 4412
curr

In [5]:
print('completed!')

completed!


Had to run several times b/c of http 502 bad gateway...

In [6]:
# now we're going to merge all the text files!
os.getcwd()

'/Users/jungwonchang/projects/web_crawling'

In [7]:
os.chdir('movies/')

In [11]:
all_text = []
for text in os.listdir():
    if '.txt' not in text:
        continue
    with open(text, 'r', encoding='utf-8') as f:
        replies = f.read()
        all_text.append(replies)

In [12]:
len(all_text)

86

In [13]:
all_text[0]

'9:fill my all holes please  \n9:동이 틀 무렵 두 사람은 서로의 대화에 변화되었다  \n6:지루해서 보다가 뛰쳐나온 몇안되는 영화  \n8:너무나 매혹적인 몰입감을 준다. 유머와 조롱이 대단하며, 그 대상이 감독 스스로에게 향하는 것을 주저하지 않는다.  \n10:셀리그먼 아저씨 왜그러셨어요.. 감독의 메시지 전달을 위해 마지막 장면이 만들어진걸로 생각하고 싶다.  \n6:유쾌한상태로 남아있고싶다면 1편만 보시길  \n7:ㅋㅋㅋ... 만고의 진리 남자는다 똑같음  \n10:1편이 사색적이고 아름답다면 2편은 지극히 현실적이고 날카롭다.  볼륨 1을 보았던 관객이라면 2편의 분위기가 다소 낯설을수 있으나 시간이 흐르면 상반된 분위기에서 전률을 느낄수 있을것이다.  \n10:재밌는데...조가 \'난 색정증환자고, 이런 내가 좋아!\' 라고 말할때의 통쾌함!!  \n10:이야기가이어지나지루하지않은...  \n2:보고싶었는데.... 자느라 못 갔어요 취소도 못 해서 돈 날렸는데 자꾸 관람평 쓰라고 해서...  \n8:외로움과 위선 그리고 섹스의 사이에서  \n10:전형적인 정신 분석학에 따른 영화. 정말 멋지다  \n10:철컥철컥철컥철컥철컥쾅  \n8:마지막장면의 반전 및 여운  \n10:인간적인 영화 인간의 내면에 대해 다시 한번 생각하게 된 영화였음  \n8:예상밖의 익살스런 구석들이 매력적. 섹스중독에 관한 뻔한 유럽식 예술영화 정도일줄 알았는데. 책을 읽는 듯한 느낌을 주는 높은 몰입, 집중, 흡입력이 매우 좋은.  \n7:꿀잼꿀잼꿀잼꿀잼꿀잼  \n8:1편보다 완성도있지만 연출은 여전히 한국인인 내 정서에너무나도 과격함  \n9:적절한 지적유희와 함께 색정증여자의 이야기는 시간가는 줄 몰랐습니다~ 즐겁게 봤습니다!  \n4:우울감 연대기 그 세번째 작품인가 화제의 작품이지만 야하지는 않다 조가 제롬과 아이까지 버리고 자기만족을 위해 떠날 땐 언제고 제롬이 다른 여자와 있으니까 싫은건가 솔직히 조가 이기적이라 생각이 들었다  \n6:감

In [14]:
with open('all_movies.txt','w', encoding='utf-8') as f:
    f.writelines(all_text)